In [110]:
import requests,json,csv,os

Movie_name = '8 mm'

API_key = '2f336ef7520a9304940fd71509ade61c'
#Use this method to return the tmdb-id (as a String) of a movie. It uses the defualt API_key which is
#set to my API_key. The input is a string of searchwords.

def get_tmdb_id_by_name(searchwords: str) -> str:
    #splits the string into each individual search word and creates query, by appending search words in query.
    words = searchwords.split(' ')
    query = 'https://api.themoviedb.org/3/search/movie/?api_key='+API_key+'&query='
    for i in range(len(words) - 1):
        query += words[i] + '+'
    query += words[len(words) - 1]

    #executes query, if response code is 200, then the query was successful
    response =  requests.get(query)
    if response.status_code==200: 
        array = response.json()
        #text contains the query as a json-file
        text = json.dumps(array)
        #print(text)
    else:
        print("No results for searchwords --> error")
        return ("No results for searchwords --> error")
    
    #dataset is json file converted into a list, containing the elements as either lists or maps
    dataset = json.loads(text)
    #extracts the movie id of the very first result
    # print(dataset)
    try:
        tmdb_id = dataset['results'][0]['id']
        return str(tmdb_id)
    except:
        return str(-1)
    # print(tmdb_id)

def get_tmdb_id_by_imdb_id(imdb_id:str) -> str:
    query = 'https://api.themoviedb.org/3/find/'+imdb_id+'?api_key='+API_key+'&external_source=imdb_id'
    response =  requests.get(query)
    if response.status_code==200:
        array = response.json()
        #text contains the query as a json-file
        text = json.dumps(array)
        #print(text)
    else:
        print("No results for searchwords --> error")
        return ("No results for searchwords --> error")
    dataset = json.loads(text)
    #extracts the movie id of the very first result
    # print(dataset)
    try:
        tmdb_id = dataset['movie_results'][0]['id']
        return str(tmdb_id)
    except:
        return str(-1)

In [111]:
import numpy as np
pg_mapping = np.genfromtxt('./baseline/imdb_id_with_age_rating_list.txt', delimiter=',', dtype=str)

def conv_nan(row):
    if row[1] == 'no age rating found':
        row[1] = 'None'
    return row

# First apply the conv() filtering on the word count
pg_mapping = np.array([conv_nan(row) for row in pg_mapping])
pg_mapping

array([['tt0032138', 'None'],
       ['tt0035423', 'PG-13'],
       ['tt0038650', 'PG'],
       ['tt0045537', 'None']], dtype='<U19')

In [112]:
#Uses tmdb id as input and returns the age certification as a String
def get_metadata_by_tmdb_id(tmdb_id: str) -> str:
    if(tmdb_id == '-1'):
        return 'metadata not found'
    else:
        query = 'https://api.themoviedb.org/3/movie/'+tmdb_id+'?api_key='+API_key
        response =  requests.get(query)
        array = response.json()
        text = json.dumps(array)
        dataset = json.loads(text)
        return dataset

In [113]:
def get_age_certfication_by_tmdb_id(tmdb_id: str) -> str:
    age_rating = ''
    if(tmdb_id != '-1'):
        query = 'https://api.themoviedb.org/3/movie/'+tmdb_id+'/release_dates?api_key='+API_key
        response =  requests.get(query)
        array = response.json()
        text = json.dumps(array)
        dataset = json.loads(text)

        list_of_certs = dataset['results']
        index = -1
        # print(dataset)
        for i in range(len(list_of_certs)):
            if(list_of_certs[i]['iso_3166_1'] == 'US'):
                index = i
                break

        if(index != -1):
            age_rating = dataset['results'][index]['release_dates'][0]['certification']
    return age_rating

In [114]:
# Test query for one movie

def map_tmdb_metadata(imdb_id: str):
    tmdb_id = get_tmdb_id_by_imdb_id(imdb_id)
    metadata = get_metadata_by_tmdb_id(tmdb_id)
    # print(json.dumps(metadata, indent=2))
    metadata = {
        "tmdb_id": metadata["id"],
        "imdb_id": metadata["imdb_id"],
        "original_title": metadata["original_title"].replace("'", ""),
        "genres": metadata["genres"],
        "tagline": metadata["tagline"].replace("'", ""),
        "overview": metadata["overview"].replace("'", ""),
        "poster_path": metadata["poster_path"],
        "pg_rating": get_age_certfication_by_tmdb_id(tmdb_id),
    }
    return metadata
    # print(json.dumps(metadata, indent=2))

In [121]:
from tqdm import tqdm
inputFile = open('./baseline/movie1K_list.txt').read().split('\n')
elastic_output_file = open("elastic_metadata_init.json", "w")

output = []
result = {}


for idx, line in tqdm(enumerate(inputFile), total=len(inputFile)):
    imdb_id = line.strip()
    metadata = map_tmdb_metadata(imdb_id)
    output.append(metadata)
    value_str = str(metadata).replace("'", '"')
    elastic_output_file.write('{"index":{"_index": "movie_metadata", "_type":"_doc", "_id": "' + str(idx + 1) + '"}}' + '\n')
    elastic_output_file.write(value_str + '\n')
    
    if len(output) == 5:
        break
result = {
    "movie_dataset": output
}

  0%|          | 4/1101 [00:05<24:28,  1.34s/it]


In [122]:
resultFile = open('./movie_metadata.json', 'w')
resultFile.write(str(result).replace("'", '"'))

2884